## Census Data Retrieval

In [2]:
import datetime


print('Last Updated On: ', datetime.datetime.now())

Last Updated On:  2022-06-21 21:32:06.117356


In [3]:
import censusdata
import pandas as pd
pd.set_option('display.max_columns', None)

In [14]:
NA_VALUE = -666666666
blockNeighborDict = {
    10001: 'Starr Hill', # 10001
    8001: 'Greenbrier', # 8001
    5012: 'Fifeville', # 5012
    7003: 'Barracks Road', # 7003
    7001: 'Barracks / Rugby', # 7001
    2012: 'Rose Hill', # 2012
    2022: 'Venable', # 2022
    4022: 'Belmont', # 4022
    3021: 'Martha Jefferson', # 3021
    5024: 'Fry\'s Spring', # 5024
    4011: 'Ridge Street', # 4011
    9001: 'Locust Grove', # 9001
    8002: 'Greenbrier', # 8002
    5021: 'Johnson Village', # 5021
    2023: 'Venable', # 2023
    8003: 'Greenbrier', # 8003
    8004: 'The Meadows', # 8004
    5013: 'Fifeville', # 5013
    5011: 'Fifeville', # 5011
    7004: 'Lewis Mountain', # 7004
    7002: 'Venable', # 7002
    5025: 'Fry\'s Spring', # 5025
    5022: 'Fry\'s Spring', # 5022
    5023: 'Fry\'s Spring', # 5023
    10002: 'North Downtown', # 10002
    10003: 'North Downtown', # 10003
    2013: 'Venable', # 2013
    2011: 'Barracks / Rugby', # 2011
    2021: '10th & Page', # 2021
    4023: 'Belmont', # 4023
    4024: 'Belmont', # 4024
    3022: 'Woolen Mills', # 3022
    4012: 'Ridge Street', # 4012
    9002: 'Locust Grove', # 9002
    6002: 'Jefferson Park Avenue', # 6002
    6001: 'Jefferson Park Avenue', # 6001
    4021: 'Belmont', # 4021
    6003: 'Jefferson Park Avenue' # 6003
}
uniqueNeighborhoods = list(set(blockNeighborDict.values()))
uniqueNeighborhoods.sort()

### Data retrieval for age:

In [8]:
censusdata.printtable(censusdata.censustable('acs5', 2019, 'B01001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B01001_001E  | SEX BY AGE                     | !! Estimate Total:                                       | int  
B01001_002E  | SEX BY AGE                     | !! !! Estimate Total: Male:                              | int  
B01001_003E  | SEX BY AGE                     | !! !! !! Estimate Total: Male: Under 5 years             | int  
B01001_004E  | SEX BY AGE                     | !! !! !! Estimate Total: Male: 5 to 9 years              | int  
B01001_005E  | SEX BY AGE                     | !! !! !! Estimate Total: Male: 10 to 14 years            | int  
B01001_006E  | SEX BY AGE                     | !! !! !! Estimate Total: Male: 15 to 17 years            | int  
B01001_007E  | SEX BY AGE                     | !! !! !! Estimate Total: Male: 18 and 19 year

In [80]:
# Function to grab data from Charlottesville City Table B01001
# In: year
# Out: dataframe containing sex by age table for overall city
def get01001DataCity(year):
    # create list of variables to pull for male column
    maleVar = ['B01001_0{:02d}E'.format(x) for x in range(3, 26)]
    # download age data for males
    dfAgeMale = censusdata.download('acs5', year, 
                                    censusdata.censusgeo([('state', '51'), 
                                                      ('county', '540')]),
                                    maleVar)
    dfAgeMale = dfAgeMale.T
    # renaming column and rows
    dfAgeMale = dfAgeMale.rename(columns={dfAgeMale.columns[0]: 'Male'},
                                 index={'B01001_003E': '0 to 4',
                                          'B01001_004E': '5 to 9',
                                          'B01001_005E': '10 to 14',
                                          'B01001_006E': '15 to 17',
                                          'B01001_007E': '18 and 19',
                                          'B01001_008E': '20',
                                          'B01001_009E': '21',
                                          'B01001_010E': '22 to 24',
                                          'B01001_011E': '25 to 29',
                                          'B01001_012E': '30 to 34',
                                          'B01001_013E': '35 to 39',
                                          'B01001_014E': '40 to 44',
                                          'B01001_015E': '45 to 49',
                                          'B01001_016E': '50 to 54',
                                          'B01001_017E': '55 to 59',
                                          'B01001_018E': '60 and 61',
                                          'B01001_019E': '62 to 64',
                                          'B01001_020E': '65 and 66',
                                          'B01001_021E': '67 to 69',
                                          'B01001_022E': '70 to 74',
                                          'B01001_023E': '75 to 79',
                                          'B01001_024E': '80 to 84', 
                                          'B01001_025E': '85 and up'})
    dfAgeMale.loc['15 to 17'] += dfAgeMale.loc['18 and 19']
    dfAgeMale.drop(['18 and 19'], inplace=True)
    dfAgeMale.loc['20'] += dfAgeMale.loc['21'] + dfAgeMale.loc['22 to 24']
    dfAgeMale.drop(['21'], inplace=True)
    dfAgeMale.drop(['22 to 24'], inplace=True)
    dfAgeMale.loc['60 and 61'] += dfAgeMale.loc['62 to 64']
    dfAgeMale.drop(['62 to 64'], inplace=True)
    dfAgeMale.loc['65 and 66'] += dfAgeMale.loc['67 to 69']
    dfAgeMale.drop(['67 to 69'], inplace=True)
    dfAgeMale = dfAgeMale.rename(index={'15 to 17': '15 to 19',
                                        '20': '20 to 24',
                                        '60 and 61': '60 to 64',
                                        '65 and 66': '65 to 69'})
    # create list of variables to pull for female column
    femaleVar = ['B01001_0{:02d}E'.format(x) for x in range(27, 50)]
    # download age data for females
    dfAgeFemale = censusdata.download('acs5', year, 
                                      censusdata.censusgeo([('state', '51'), 
                                                      ('county', '540')]),
                                      femaleVar)
    dfAgeFemale = dfAgeFemale.T
    # renaming column and rows
    dfAgeFemale = dfAgeFemale.rename(columns={dfAgeFemale.columns[0]: 'Female'},
                                     index={'B01001_027E': '0 to 4',
                                          'B01001_028E': '5 to 9',
                                          'B01001_029E': '10 to 14',
                                          'B01001_030E': '15 to 17',
                                          'B01001_031E': '18 and 19',
                                          'B01001_032E': '20',
                                          'B01001_033E': '21',
                                          'B01001_034E': '22 to 24',
                                          'B01001_035E': '25 to 29',
                                          'B01001_036E': '30 to 34',
                                          'B01001_037E': '35 to 39',
                                          'B01001_038E': '40 to 44',
                                          'B01001_039E': '45 to 49',
                                          'B01001_040E': '50 to 54',
                                          'B01001_041E': '55 to 59',
                                          'B01001_042E': '60 and 61',
                                          'B01001_043E': '62 to 64',
                                          'B01001_044E': '65 and 66',
                                          'B01001_045E': '67 to 69',
                                          'B01001_046E': '70 to 74',
                                          'B01001_047E': '75 to 79',
                                          'B01001_048E': '80 to 84', 
                                          'B01001_049E': '85 and up'})
    dfAgeFemale.loc['15 to 17'] += dfAgeFemale.loc['18 and 19']
    dfAgeFemale.drop(['18 and 19'], inplace=True)
    dfAgeFemale.loc['20'] += dfAgeFemale.loc['21'] + dfAgeFemale.loc['22 to 24']
    dfAgeFemale.drop(['21'], inplace=True)
    dfAgeFemale.drop(['22 to 24'], inplace=True)
    dfAgeFemale.loc['60 and 61'] += dfAgeFemale.loc['62 to 64']
    dfAgeFemale.drop(['62 to 64'], inplace=True)
    dfAgeFemale.loc['65 and 66'] += dfAgeFemale.loc['67 to 69']
    dfAgeFemale.drop(['67 to 69'], inplace=True)
    dfAgeFemale = dfAgeFemale.rename(index={'15 to 17': '15 to 19',
                                        '20': '20 to 24',
                                        '60 and 61': '60 to 64',
                                        '65 and 66': '65 to 69'})
    dfAge = dfAgeMale
    dfAge['Female'] = dfAgeFemale['Female']
    # get total population count
    totalPop = dfAge['Male'].sum() + dfAge['Female'].sum()
    # get pct df from counts
    dfAgePct = dfAge.apply(lambda x: 100 * x / float(totalPop))
    dfAgeFull = pd.concat([dfAgePct, dfAge], axis=1, join='inner')
    dfAgeFull.columns.values[2:] = dfAgeFull.columns.values[2:] + 'ct'
    dfAgeFull['Year'] = year

    return dfAgeFull

In [23]:
def get01001Historical(years, hood = False):
    for year in years:
        if hood:
            if year == years[0]:
                df = get01001DataHood(year)
            else:
                df = pd.concat([df, get01001DataHood(year)])
        else:
            if year == years[0]:
                df = get01001DataCity(year)
            else:
                df = pd.concat([df, get01001DataCity(year)])
    df.index.name = 'Age'
    return df

In [82]:
dfAge = get01001Historical([year for year in range(2009, 2021)])

In [83]:
dfAge

,Male,Female,Malect,Femalect,Year
Age,,,,,
0 to 4,2.754089,2.708187,1140,1121,2009
5 to 9,2.488343,2.256420,1030,934,2009
10 to 14,2.019665,1.659701,836,687,2009
15 to 19,3.660039,4.177035,1515,1729,2009
20 to 24,11.192714,12.154229,4633,5031,2009
...,...,...,...,...,...
65 to 69,1.785374,2.325434,843,1098,2020
70 to 74,1.529110,1.673126,722,790,2020
75 to 79,0.724315,0.989051,342,467,2020


dfAge.to_csv('data/ageCity.csv')

In [21]:
# Function to grab data from Charlottesville City Table B01001
# In: year
# Out: dataframe containing sex by age table for neighborhoods
def get01001DataHood(year):
    # create list of variables to pull for male column
    maleVar = ['B01001_0{:02d}E'.format(x) for x in range(3, 26)]
    # download age data for males
    dfAgeMale = censusdata.download('acs5', year, 
                                    censusdata.censusgeo([('state', '51'), 
                                                          ('county', '540'),
                                                          ('block group', '*')]),
                                    maleVar)
    # add neighborhoods for grouping
    dfAgeMale['Neighborhood'] = ''
    for index, row in dfAgeMale.iterrows():
        if index.geo[2][1][2]:
            # 100 census tract case
            dfAgeMale.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][2:] + \
                                                                         index.geo[3][1])]
        else:
            # 0xx census tract case
            dfAgeMale.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][3:] + \
                                                                         index.geo[3][1])]
    # remove nan values and sum neighborhood counts
    dfAgeMale = dfAgeMale.groupby('Neighborhood').sum().astype(int)
    # transpose
    dfAgeMale = dfAgeMale.T
    # renaming rows
    dfAgeMale = dfAgeMale.rename(index={'B01001_003E': '0 to 4',
                                        'B01001_004E': '5 to 9',
                                        'B01001_005E': '10 to 14',
                                        'B01001_006E': '15 to 17',
                                        'B01001_007E': '18 and 19',
                                        'B01001_008E': '20',
                                        'B01001_009E': '21',
                                        'B01001_010E': '22 to 24',
                                        'B01001_011E': '25 to 29',
                                        'B01001_012E': '30 to 34',
                                        'B01001_013E': '35 to 39',
                                        'B01001_014E': '40 to 44',
                                        'B01001_015E': '45 to 49',
                                        'B01001_016E': '50 to 54',
                                        'B01001_017E': '55 to 59',
                                        'B01001_018E': '60 and 61',
                                        'B01001_019E': '62 to 64',
                                        'B01001_020E': '65 and 66',
                                        'B01001_021E': '67 to 69',
                                        'B01001_022E': '70 to 74',
                                        'B01001_023E': '75 to 79',
                                        'B01001_024E': '80 to 84', 
                                        'B01001_025E': '85 and up'})
    # combining some age groups to have even sizes
    dfAgeMale.loc['15 to 17'] += dfAgeMale.loc['18 and 19']
    dfAgeMale.drop(['18 and 19'], inplace=True)
    dfAgeMale.loc['20'] += dfAgeMale.loc['21'] + dfAgeMale.loc['22 to 24']
    dfAgeMale.drop(['21'], inplace=True)
    dfAgeMale.drop(['22 to 24'], inplace=True)
    dfAgeMale.loc['60 and 61'] += dfAgeMale.loc['62 to 64']
    dfAgeMale.drop(['62 to 64'], inplace=True)
    dfAgeMale.loc['65 and 66'] += dfAgeMale.loc['67 to 69']
    dfAgeMale.drop(['67 to 69'], inplace=True)
    # renaming combined groups
    dfAgeMale = dfAgeMale.rename(index={'15 to 17': '15 to 19',
                                        '20': '20 to 24',
                                        '60 and 61': '60 to 64',
                                        '65 and 66': '65 to 69'})
    # create list of variables to pull for female column
    femaleVar = ['B01001_0{:02d}E'.format(x) for x in range(27, 50)]
    # download age data for females
    dfAgeFemale = censusdata.download('acs5', year, 
                                      censusdata.censusgeo([('state', '51'), 
                                                            ('county', '540'),
                                                            ('block group', '*')]),
                                      femaleVar)
    # add neighborhoods for grouping
    dfAgeFemale['Neighborhood'] = ''
    for index, row in dfAgeFemale.iterrows():
        if index.geo[2][1][2]:
            # 100 census tract case
            dfAgeFemale.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][2:] + \
                                                                           index.geo[3][1])]
        else:
            # 0xx census tract case
            dfAgeFemale.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][3:] + \
                                                                           index.geo[3][1])]
    # remove nan values and sum neighborhood counts
    dfAgeFemale = dfAgeFemale.groupby('Neighborhood').sum().astype(int)
    # transpose
    dfAgeFemale = dfAgeFemale.T
    # renaming column and rows
    dfAgeFemale = dfAgeFemale.rename(index={'B01001_027E': '0 to 4',
                                            'B01001_028E': '5 to 9',
                                            'B01001_029E': '10 to 14',
                                            'B01001_030E': '15 to 17',
                                            'B01001_031E': '18 and 19',
                                            'B01001_032E': '20',
                                            'B01001_033E': '21',
                                            'B01001_034E': '22 to 24',
                                            'B01001_035E': '25 to 29',
                                            'B01001_036E': '30 to 34',
                                            'B01001_037E': '35 to 39',
                                            'B01001_038E': '40 to 44',
                                            'B01001_039E': '45 to 49',
                                            'B01001_040E': '50 to 54',
                                            'B01001_041E': '55 to 59',
                                            'B01001_042E': '60 and 61',
                                            'B01001_043E': '62 to 64',
                                            'B01001_044E': '65 and 66',
                                            'B01001_045E': '67 to 69',
                                            'B01001_046E': '70 to 74',
                                            'B01001_047E': '75 to 79',
                                            'B01001_048E': '80 to 84', 
                                            'B01001_049E': '85 and up'})
    # combining some age groups to have even sizes
    dfAgeFemale.loc['15 to 17'] += dfAgeFemale.loc['18 and 19']
    dfAgeFemale.drop(['18 and 19'], inplace=True)
    dfAgeFemale.loc['20'] += dfAgeFemale.loc['21'] + dfAgeFemale.loc['22 to 24']
    dfAgeFemale.drop(['21'], inplace=True)
    dfAgeFemale.drop(['22 to 24'], inplace=True)
    dfAgeFemale.loc['60 and 61'] += dfAgeFemale.loc['62 to 64']
    dfAgeFemale.drop(['62 to 64'], inplace=True)
    dfAgeFemale.loc['65 and 66'] += dfAgeFemale.loc['67 to 69']
    dfAgeFemale.drop(['67 to 69'], inplace=True)
    # renaming combined groups
    dfAgeFemale = dfAgeFemale.rename(index={'15 to 17': '15 to 19',
                                            '20': '20 to 24',
                                            '60 and 61': '60 to 64',
                                            '65 and 66': '65 to 69'})
    # join male/female datasets and add variable identifier
    dfAge = pd.concat([dfAgeMale, dfAgeFemale], axis=1, join='inner')
    dfAge.columns.values[:19] = dfAge.columns.values[:19] + '_M'
    dfAge.columns.values[19:] = dfAge.columns.values[19:] + '_F'
    # generating pct df
    dfAgePct = dfAge
    for hood in uniqueNeighborhoods:
        totalPop = dfAge[hood + "_M"].sum() + dfAge[hood + "_F"].sum()
        dfAgePct[hood + "_M"] = 100 * dfAgePct[hood + "_M"] / float(totalPop)
        dfAgePct[hood + "_F"] = 100 * dfAgePct[hood + "_F"] / float(totalPop)
    # combining pct and ct df
    dfAgeFull = pd.concat([dfAgePct, dfAge], axis=1, join='inner')
    dfAgeFull.columns.values[38:] = dfAgeFull.columns.values[38:] + "ct"
    # add year
    dfAgeFull['Year'] = year

    return dfAgeFull

In [24]:
dfAgeHood = get01001Historical([year for year in range(2013, 2021)], True)

In [25]:
dfAgeHood

Neighborhood,10th & Page_M,Barracks / Rugby_M,Barracks Road_M,Belmont_M,Fifeville_M,Fry's Spring_M,Greenbrier_M,Jefferson Park Avenue_M,Johnson Village_M,Lewis Mountain_M,Locust Grove_M,Martha Jefferson_M,North Downtown_M,Ridge Street_M,Rose Hill_M,Starr Hill_M,The Meadows_M,Venable_M,Woolen Mills_M,10th & Page_F,Barracks / Rugby_F,Barracks Road_F,Belmont_F,Fifeville_F,Fry's Spring_F,Greenbrier_F,Jefferson Park Avenue_F,Johnson Village_F,Lewis Mountain_F,Locust Grove_F,Martha Jefferson_F,North Downtown_F,Ridge Street_F,Rose Hill_F,Starr Hill_F,The Meadows_F,Venable_F,Woolen Mills_F,10th & Page_Mct,Barracks / Rugby_Mct,Barracks Road_Mct,Belmont_Mct,Fifeville_Mct,Fry's Spring_Mct,Greenbrier_Mct,Jefferson Park Avenue_Mct,Johnson Village_Mct,Lewis Mountain_Mct,Locust Grove_Mct,Martha Jefferson_Mct,North Downtown_Mct,Ridge Street_Mct,Rose Hill_Mct,Starr Hill_Mct,The Meadows_Mct,Venable_Mct,Woolen Mills_Mct,10th & Page_Fct,Barracks / Rugby_Fct,Barracks Road_Fct,Belmont_Fct,Fifeville_Fct,Fry's Spring_Fct,Greenbrier_Fct,Jefferson Park Avenue_Fct,Johnson Village_Fct,Lewis Mountain_Fct,Locust Grove_Fct,Martha Jefferson_Fct,North Downtown_Fct,Ridge Street_Fct,Rose Hill_Fct,Starr Hill_Fct,The Meadows_Fct,Venable_Fct,Woolen Mills_Fct,Year
Age,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0 to 4,0.349243,4.497099,0.000000,2.617535,3.445354,3.404122,3.163993,0.468436,8.208092,0.000000,6.563877,2.681159,3.827038,3.115547,2.470356,0.000000,4.709937,0.967600,1.626016,5.355064,2.901354,5.287356,1.728081,2.569896,2.498438,3.431373,0.468436,5.317919,0.000000,2.555066,7.608696,3.628231,4.597701,0.000000,0.000000,5.916140,0.689049,3.150407,0.349243,4.497099,0.000000,2.617535,3.445354,3.404122,3.163993,0.468436,8.208092,0.000000,6.563877,2.681159,3.827038,3.115547,2.470356,0.000000,4.709937,0.967600,1.626016,5.355064,2.901354,5.287356,1.728081,2.569896,2.498438,3.431373,0.468436,5.317919,0.000000,2.555066,7.608696,3.628231,4.597701,0.000000,0.000000,5.916140,0.689049,3.150407,2013
5 to 9,4.307334,2.417795,0.000000,3.761118,3.586557,0.000000,2.852050,0.133839,3.468208,0.000000,1.365639,1.811594,1.491054,4.053237,2.470356,1.334951,7.811603,0.982261,3.252033,3.026775,4.545455,0.000000,1.194409,2.005083,0.000000,3.297683,0.133839,1.098266,0.000000,1.233480,3.405797,3.031809,2.813067,0.000000,1.456311,1.665709,0.542442,1.016260,4.307334,2.417795,0.000000,3.761118,3.586557,0.000000,2.852050,0.133839,3.468208,0.000000,1.365639,1.811594,1.491054,4.053237,2.470356,1.334951,7.811603,0.982261,3.252033,3.026775,4.545455,0.000000,1.194409,2.005083,0.000000,3.297683,0.133839,1.098266,0.000000,1.233480,3.405797,3.031809,2.813067,0.000000,1.456311,1.665709,0.542442,1.016260,2013
10 to 14,0.000000,0.967118,0.000000,1.601017,4.264332,1.249219,0.668449,0.401517,0.751445,0.000000,1.585903,2.898551,1.838966,4.809437,0.691700,0.000000,4.422746,0.557103,1.422764,3.608847,3.094778,1.379310,1.677255,2.993505,1.093067,4.233512,0.401517,5.491329,0.000000,2.731278,1.739130,2.087475,1.391410,4.150198,0.000000,1.665709,0.469139,7.418699,0.000000,0.967118,0.000000,1.601017,4.264332,1.249219,0.668449,0.401517,0.751445,0.000000,1.585903,2.898551,1.838966,4.809437,0.691700,0.000000,4.422746,0.557103,1.422764,3.608847,3.094778,1.379310,1.677255,2.993505,1.093067,4.233512,0.401517,5.491329,0.000000,2.731278,1.739130,2.087475,1.391410,4.150198,0.000000,1.665709,0.469139,7.418699,2013
15 to 19,4.656577,2.417795,5.287356,3.405337,3.897204,1.374141,2.005348,10.617890,5.202312,4.044118,4.581498,2.971014,0.646123,4.204477,0.000000,1.213592,4.997128,8.136637,1.321138,0.000000,2.079304,15.862069,1.855146,1.920361,2.560899,1.604278,7.539594,0.867052,9.436275,0.704846,0.000000,1.242545,2.329099,0.592885,0.728155,0.919012,5.145873,0.000000,4.656577,2.417795,5.287356,3.405337,3.897204,1.374141,2.005348,10.617890,5.202312,4.044118,4.581498,2.971014,0.646123,4.204477,0.000000,1.213592,4.997128,8.136637,1.321138,0.000000,2.079304,15.862069,1.855146,1.920361,2.560899,1.6

dfAgeHood.to_csv('data/ageNeighborhood.csv')

### Data retrieval for race and ethnicity:

In [4]:
censusdata.printtable(censusdata.censustable('acs5', 2019, 'B03002'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B03002_001E  | HISPANIC OR LATINO ORIGIN BY R | !! Estimate Total:                                       | int  
B03002_002E  | HISPANIC OR LATINO ORIGIN BY R | !! !! Estimate Total: Not Hispanic or Latino:            | int  
B03002_003E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: White a | int  
B03002_004E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Black o | int  
B03002_005E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: America | int  
B03002_006E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Asian a | int  
B03002_007E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Lati

In [5]:
# Function to grab data from Charlottesville City Table B03002
# In: year
# Out: dataframe containing race and ethnicity table for overall city
def get03002DataCity(year):
    # create list of variables to pull
    varList = ['B03002_0{:02d}E'.format(x) for x in [3, 4, 5, 6, 7, 8, 9, 12]]
    # download race data
    df = censusdata.download('acs5', year, 
                             censusdata.censusgeo([('state', '51'), 
                                                   ('county', '540')]),
                             varList)
    df = df.T
    # renaming column and rows
    df = df.rename(columns={df.columns[0]: 'Pop'},
                   index={'B03002_003E': 'White, not Hispanic',
                          'B03002_004E': 'Black, not Hispanic',
                          'B03002_005E': 'American Indian, not Hispanic',
                          'B03002_006E': 'Asian, not Hispanic',
                          'B03002_007E': 'Pacific Islander, not Hispanic',
                          'B03002_008E': 'Some other race, not Hispanic',
                          'B03002_009E': 'Two or more races, not Hispanic',
                          'B03002_012E': 'Hispanic or Latino (may be of any race)'})
    # get total population count
    totalPop = df['Pop'].sum()
    # get pct df from counts
    dfPct = df.apply(lambda x: 100 * x / float(totalPop))
    dfFull = pd.concat([dfPct, df], axis=1, join='inner')
    # add identifier to count variable
    dfFull.columns.values[1] = dfFull.columns.values[1] + 'ct'
    # add year col
    dfFull['Year'] = year

    return dfFull

In [6]:
def get03002Historical(years, hood = False):
    for year in years:
        if hood:
            if year == years[0]:
                df = get03002DataHood(year)
            else:
                df = pd.concat([df, get03002DataHood(year)])
        else:
            if year == years[0]:
                df = get03002DataCity(year)
            else:
                df = pd.concat([df, get03002DataCity(year)])
    if not hood:
        ag = [f'({df.iloc[i, 1]:,} : {df.iloc[i, 0]:.2f}%)'
              for i in range(df.shape[0])]
        df['ag'] = ag
    df.index.name = 'Race and Ethnicity'
    return df

In [8]:
dfEthnicity = get03002Historical([year for year in range(2009, 2021)])

In [9]:
dfEthnicity

,Pop,Popct,Year,ag
Race and Ethnicity,,,,
"White, not Hispanic",69.267751,28672,2009,"(28,672 : 69.27%)"
"Black, not Hispanic",19.885005,8231,2009,"(8,231 : 19.89%)"
"American Indian, not Hispanic",0.074892,31,2009,(31 : 0.07%)
"Asian, not Hispanic",4.392047,1818,2009,"(1,818 : 4.39%)"
"Pacific Islander, not Hispanic",0.028990,12,2009,(12 : 0.03%)
...,...,...,...,...
"Asian, not Hispanic",7.016541,3313,2020,"(3,313 : 7.02%)"
"Pacific Islander, not Hispanic",0.010589,5,2020,(5 : 0.01%)
"Some other race, not Hispanic",0.332507,157,2020,(157 : 0.33%)


dfEthnicity.to_csv('data/raceEthnicityCity.csv')

In [11]:
# Function to grab data from Charlottesville City Table B03002
# In: year
# Out: dataframe containing race table for neighborhoods
def get03002DataHood(year):
    # create list of variables to pull
    varList = ['B03002_0{:02d}E'.format(x) for x in [3, 4, 5, 6, 7, 8, 9, 12]]
    # download race data
    df = censusdata.download('acs5', year, 
                             censusdata.censusgeo([('state', '51'), 
                                                   ('county', '540'),
                                                   ('block group', '*')]),
                             varList)
    # add neighborhoods for grouping
    df['Neighborhood'] = ''
    for index, row in df.iterrows():
        if index.geo[2][1][2]:
            # 100 census tract case
            df.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][2:] + \
                                                                  index.geo[3][1])]
        else:
            # 0xx census tract case
            df.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][3:] + \
                                                                  index.geo[3][1])]
    # remove nan values and sum neighborhood counts
    df = df.groupby('Neighborhood').sum().astype(int)
    # transpose
    df = df.T
    # renaming column and rows
    df = df.rename(index={'B03002_003E': 'White, not Hispanic',
                          'B03002_004E': 'Black, not Hispanic',
                          'B03002_005E': 'American Indian, not Hispanic',
                          'B03002_006E': 'Asian, not Hispanic',
                          'B03002_007E': 'Pacific Islander, not Hispanic',
                          'B03002_008E': 'Some other race, not Hispanic',
                          'B03002_009E': 'Two or more races, not Hispanic',
                          'B03002_012E': 'Hispanic or Latino (may be of any race)'})
    dfPct = df.apply(lambda x: 100 * x / float(sum(x)))
    dfFull = pd.concat([dfPct, df], axis=1, join='inner')
    dfFull.columns.values[19:] = dfFull.columns.values[19:] + 'ct'
    dfFull['Year'] = year

    return dfFull

In [15]:
dfRaceEthnicity = get03002Historical([year for year in range(2013, 2021)], True)

In [16]:
dfRaceEthnicity

Neighborhood,10th & Page,Barracks / Rugby,Barracks Road,Belmont,Fifeville,Fry's Spring,Greenbrier,Jefferson Park Avenue,Johnson Village,Lewis Mountain,Locust Grove,Martha Jefferson,North Downtown,Ridge Street,Rose Hill,Starr Hill,The Meadows,Venable,Woolen Mills,10th & Pagect,Barracks / Rugbyct,Barracks Roadct,Belmontct,Fifevillect,Fry's Springct,Greenbrierct,Jefferson Park Avenuect,Johnson Villagect,Lewis Mountainct,Locust Grovect,Martha Jeffersonct,North Downtownct,Ridge Streetct,Rose Hillct,Starr Hillct,The Meadowsct,Venablect,Woolen Millsct,Year
Race and Ethnicity,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"White, not Hispanic",19.906868,70.551257,76.091954,78.653113,33.267439,79.825109,90.730838,65.335713,54.624277,67.524510,85.066079,75.072464,87.276342,40.562613,52.865613,82.766990,43.653073,73.215071,76.930894,171,1459,331,3095,1178,2556,2036,2929,945,551,1931,1036,1756,1341,535,682,760,4994,757,2013
"Black, not Hispanic",76.600698,13.974855,3.448276,16.365947,51.793279,11.961274,2.985740,5.264332,31.445087,3.063725,4.273128,12.971014,2.783300,47.670901,41.600791,15.291262,25.674899,10.233104,21.036585,658,289,15,644,1834,383,67,236,544,25,97,179,56,1576,421,126,447,698,207,2013
"American Indian, not Hispanic",0.000000,0.483559,0.000000,0.000000,0.000000,0.000000,0.000000,1.204551,0.000000,0.000000,0.000000,0.000000,1.242545,0.000000,0.000000,0.000000,0.000000,0.205248,0.000000,0,10,0,0,0,0,0,54,0,0,0,0,25,0,0,0,0,14,0,2013
"Asian, not Hispanic",1.047730,3.916828,19.310345,1.880559,6.438859,4.434728,0.534759,17.443676,9.710983,20.343137,0.748899,4.565217,0.745527,0.967937,5.533597,0.000000,15.795520,10.570298,0.813008,9,81,84,74,228,142,12,782,168,166,17,63,15,32,56,0,275,721,8,2013
"Pacific Islander, not Hispanic",0.000000,0.000000,0.000000,0.000000,0.706015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.175927,0.000000,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,12,0,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Asian, not Hispanic",9.947984,0.511654,15.483871,1.015228,6.030151,8.082104,2.007648,28.087370,3.948667,7.218684,5.998126,3.216861,1.509017,4.410399,1.207729,1.015228,10.562310,9.344216,0.300752,153,9,144,42,276,252,42,1003,80,68,128,58,41,190,10,10,139,664,4,2020
"Pacific Islander, not Hispanic",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.277316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,2020
"Some other race, not Hispanic",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.908222,0.000000,0.000000,0.000000,0.000000,0.554631,0.515274,0.000000,0.000000,0.000000,0.000000,1.604278,0.000000,0,0,0,0,0,0,19,0,0,0,0,10,14,0,0,0,0,114,0,2020


dfRaceEthnicity.to_csv('data/raceEthnicityNeighborhood.csv')

### Data retrieval for income:

In [126]:
censusdata.printtable(censusdata.censustable('acs5', 2019, 'B19001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19001_001E  | HOUSEHOLD INCOME IN THE PAST 1 | !! Estimate Total:                                       | int  
B19001_002E  | HOUSEHOLD INCOME IN THE PAST 1 | !! !! Estimate Total: Less than $10,000                  | int  
B19001_003E  | HOUSEHOLD INCOME IN THE PAST 1 | !! !! Estimate Total: $10,000 to $14,999                 | int  
B19001_004E  | HOUSEHOLD INCOME IN THE PAST 1 | !! !! Estimate Total: $15,000 to $19,999                 | int  
B19001_005E  | HOUSEHOLD INCOME IN THE PAST 1 | !! !! Estimate Total: $20,000 to $24,999                 | int  
B19001_006E  | HOUSEHOLD INCOME IN THE PAST 1 | !! !! Estimate Total: $25,000 to $29,999                 | int  
B19001_007E  | HOUSEHOLD INCOME IN THE PAST 1 | !! !! Estimate Total: $30,000 to $34,999     

In [9]:
# Function to grab data from Charlottesville City Table B19001
# In: year
# Out: dataframe containing income table for overall city
def get19001DataCity(year):
    # create list of variables to pull
    varList = ['B19001_0{:02d}E'.format(x) for x in range(2, 18)]
    # download income data
    df = censusdata.download('acs5', year, 
                             censusdata.censusgeo([('state', '51'), 
                                                   ('county', '540')]),
                             varList)
    df = df.T
    # renaming column and rows
    df = df.rename(columns={df.columns[0]: 'Income'},
                   index={'B19001_002E': 'Less than $10,000',
                          'B19001_003E': '$10,000 to 14,999',
                          'B19001_004E': '$15,000 to 19,999',
                          'B19001_005E': '$20,000 to 24,999',
                          'B19001_006E': '$25,000 to 29,999',
                          'B19001_007E': '$30,000 to 34,999',
                          'B19001_008E': '$35,000 to 39,999', 
                          'B19001_009E': '$40,000 to 44,999', 
                          'B19001_010E': '$45,000 to 49,999', 
                          'B19001_011E': '$50,000 to 59,999', 
                          'B19001_012E': '$60,000 to 74,999', 
                          'B19001_013E': '$75,000 to 99,999', 
                          'B19001_014E': '$100,000 to 124,999', 
                          'B19001_015E': '$125,000 to 149,999', 
                          'B19001_016E': '$150,000 to 199,999', 
                          'B19001_017E': '$200,000 or more'})
    # get total population count
    totalPop = df['Income'].sum()
    # get pct df from counts
    dfPct = df.apply(lambda x: 100 * x / float(totalPop))
    dfFull = pd.concat([dfPct, df], axis=1, join='inner')
    dfFull.columns.values[1] = dfFull.columns.values[1] + 'ct'
    dfFull['Year'] = year

    return dfFull

In [10]:
def get19001Historical(years, hood = False):
    for year in years:
        if hood:
            if year == years[0]:
                df = get19001DataHood(year)
            else:
                df = pd.concat([df, get19001DataHood(year)])
        else:
            if year == years[0]:
                df = get19001DataCity(year)
            else:
                df = pd.concat([df, get19001DataCity(year)])
    if not hood:
        ag = [f'({df.iloc[i, 1]:,} : {df.iloc[i, 0]:.2f}%)'
              for i in range(df.shape[0])]
        df['ag'] = ag
    df.index.name = 'Bracket'
    return df

In [11]:
dfIncome = get19001Historical([year for year in range(2009, 2021)])

In [12]:
dfIncome

,Income,Incomect,Year,ag
Bracket,,,,
"Less than $10,000",15.618947,2661,2009,"(2,661 : 15.62%)"
"$10,000 to 14,999",6.145448,1047,2009,"(1,047 : 6.15%)"
"$15,000 to 19,999",6.239361,1063,2009,"(1,063 : 6.24%)"
"$20,000 to 24,999",9.091976,1549,2009,"(1,549 : 9.09%)"
"$25,000 to 29,999",4.736749,807,2009,(807 : 4.74%)
...,...,...,...,...
"$75,000 to 99,999",12.267460,2308,2020,"(2,308 : 12.27%)"
"$100,000 to 124,999",8.408632,1582,2020,"(1,582 : 8.41%)"
"$125,000 to 149,999",5.798873,1091,2020,"(1,091 : 5.80%)"


dfIncome.to_csv('data/incomeCity.csv')

In [40]:
# Function to grab data from Charlottesville City Table B19001
# In: year
# Out: dataframe containing income table for neighborhoods
def get19001DataHood(year):
    # create list of variables to pull
    varList = ['B19001_0{:02d}E'.format(x) for x in range(2, 18)]
    # download income data
    df = censusdata.download('acs5', year, 
                             censusdata.censusgeo([('state', '51'), 
                                                   ('county', '540'),
                                                   ('block group', '*')]),
                             varList)
    # add neighborhoods for grouping
    df['Neighborhood'] = ''
    for index, row in df.iterrows():
        if index.geo[2][1][2]:
            # 100 census tract case
            df.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][2:] + \
                                                                  index.geo[3][1])]
        else:
            # 0xx census tract case
            df.loc[index, 'Neighborhood'] = blockNeighborDict[int(index.geo[2][1][3:] + \
                                                                  index.geo[3][1])]
    # remove nan values and sum neighborhood counts
    df = df.groupby('Neighborhood').sum().astype(int)
    # transpose
    df = df.T
    # renaming column and rows
    df = df.rename(index={'B19001_002E': 'Less than $10,000',
                          'B19001_003E': '$10,000 to 14,999',
                          'B19001_004E': '$15,000 to 19,999',
                          'B19001_005E': '$20,000 to 24,999',
                          'B19001_006E': '$25,000 to 29,999',
                          'B19001_007E': '$30,000 to 34,999',
                          'B19001_008E': '$35,000 to 39,999', 
                          'B19001_009E': '$40,000 to 44,999', 
                          'B19001_010E': '$45,000 to 49,999', 
                          'B19001_011E': '$50,000 to 59,999', 
                          'B19001_012E': '$60,000 to 74,999', 
                          'B19001_013E': '$75,000 to 99,999', 
                          'B19001_014E': '$100,000 to 124,999', 
                          'B19001_015E': '$125,000 to 149,999', 
                          'B19001_016E': '$150,000 to 199,999', 
                          'B19001_017E': '$200,000 or more'})
    dfPct = df.apply(lambda x: 100 * x / float(sum(x)))
    dfFull = pd.concat([dfPct, df], axis=1, join='inner')
    dfFull.columns.values[19:] = dfFull.columns.values[19:] + 'ct'
    dfFull['Year'] = year

    return dfFull

In [42]:
dfIncomeHist = get19001Historical([year for year in range(2013, 2021)], True)

In [43]:
dfIncomeHist

Neighborhood,10th & Page,Barracks / Rugby,Barracks Road,Belmont,Fifeville,Fry's Spring,Greenbrier,Jefferson Park Avenue,Johnson Village,Lewis Mountain,Locust Grove,Martha Jefferson,North Downtown,Ridge Street,Rose Hill,Starr Hill,The Meadows,Venable,Woolen Mills,10th & Pagect,Barracks / Rugbyct,Barracks Roadct,Belmontct,Fifevillect,Fry's Springct,Greenbrierct,Jefferson Park Avenuect,Johnson Villagect,Lewis Mountainct,Locust Grovect,Martha Jeffersonct,North Downtownct,Ridge Streetct,Rose Hillct,Starr Hillct,The Meadowsct,Venablect,Woolen Millsct,Year
Bracket,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Less than $10,000",21.460177,9.081633,19.200000,11.011591,11.988514,7.676903,0.000000,34.869326,7.226891,29.914530,2.436647,4.360465,0.597015,22.996255,21.529745,9.302326,32.220367,40.626632,12.142857,97,89,24,209,167,115,0,507,43,70,25,30,6,307,76,52,193,778,51,2013
"$10,000 to 14,999",8.407080,2.857143,8.800000,7.218124,5.096913,5.740988,0.765864,3.782669,0.000000,3.418803,2.046784,13.808140,2.786070,2.846442,2.549575,6.440072,3.505843,3.812010,0.000000,38,28,11,137,71,86,7,55,0,8,21,95,28,38,9,36,21,73,0,2013
"$15,000 to 19,999",11.504425,0.000000,24.800000,6.375132,4.737976,0.000000,2.844639,5.914718,2.521008,0.000000,4.483431,2.034884,3.681592,7.565543,5.099150,4.114490,9.015025,8.093995,5.714286,52,0,31,121,66,0,26,86,15,0,46,14,37,101,18,23,54,155,24,2013
"$20,000 to 24,999",18.362832,2.244898,6.400000,11.116965,6.748026,4.205607,1.312910,3.026135,6.050420,2.136752,1.949318,2.616279,7.562189,1.573034,15.580737,7.155635,5.843072,3.603133,1.428571,83,22,8,211,94,63,12,44,36,5,20,18,76,21,55,40,35,69,6,2013
"$25,000 to 29,999",7.964602,3.163265,5.600000,4.004215,8.470926,7.610147,2.078775,10.041265,0.000000,0.000000,6.432749,3.779070,5.870647,6.516854,0.000000,6.082290,8.347245,1.096606,1.904762,36,31,7,76,118,114,19,146,0,0,66,26,59,87,0,34,50,21,8,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"$75,000 to 99,999",19.026549,15.107914,9.352518,11.231203,12.244898,14.538153,8.997722,7.450980,7.994924,6.349206,15.610860,18.154762,20.872274,10.740993,4.395604,12.166172,7.504363,5.645534,31.886477,86,105,39,239,204,181,79,95,63,16,138,183,268,158,16,82,43,122,191,2020
"$100,000 to 124,999",2.654867,4.460432,6.954436,11.372180,8.643457,11.646586,25.284738,1.411765,6.218274,7.936508,9.615385,4.563492,7.943925,6.866077,2.197802,21.810089,15.706806,2.313744,6.844741,12,31,29,242,144,145,222,18,49,20,85,46,102,101,8,147,90,50,41,2020
"$125,000 to 149,999",9.070796,6.187050,2.637890,3.054511,5.042017,12.128514,5.239180,3.058824,12.309645,6.349206,7.692308,4.861111,7.943925,5.846363,0.000000,3.709199,8.726003,3.100416,8.514190,41,43,11,65,84,151,46,39,97,16,68,49,102,86,0,25,50,67,51,2020


dfIncomeHist.to_csv('data/incomeNeighborhood.csv')